ABAG Landuse Calculations

This notebook documents the joining data between VTA's TAZ Zones and 


In [ ]:
#This uses geopy36
import geopandas as gpd
import pandas as pd
%matplotlib inline  
from shapely.geometry import Point

from simpledbf import Dbf5
#pip install simpledbf
#https://pypi.python.org/pypi/simpledbf/0.2.4

vta_taz = gpd.read_file('VTA_TAZ/')
vta_taz = vta_taz.rename(columns={'TAZ':'VTA_TAZ'})

dbf = Dbf5('2017ABAGLanduseAllocation/VTA/zmast13.dbf')
vta_dbf = dbf.to_dataframe()
vta_dbf = vta_dbf.rename(columns={'ZONE':'VTA_TAZ'})

vta_taz = pd.merge(vta_taz,vta_dbf)


# mtc_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/MTC/MTCTAZ1454.dbf')


#http://analytics.mtc.ca.gov/foswiki/UrbanSimTwo/OutputToTravelModel

PyTables is not installed. No support for HDF output.


In [ ]:
mtc_taz = gpd.read_file('MTC/')
mtc_taz = mtc_taz.rename(columns={'TAZ1454':'ZONE'})
abag_2010 = pd.read_csv('2017ABAGLanduseAllocation/ABAGPROJ2017/DRAFT/run7224c_taz_summaries_2010.csv')
mtc_taz = pd.merge(mtc_taz,abag_2010)

In [ ]:
mtc_taz.head()

In [ ]:
# vta_taz['centroid'] = vta_taz.centroid
# vta_taz = vta_taz.set_geometry('centroid')
# vta_taz['old_geometry'] = vta_taz['geometry']
# vta_taz['geometry'] = vta_taz['centroid']

# centroid = gpd.sjoin(mtc_taz[['ZONE','geometry']], vta_taz, how = "right", op='contains')
# centroid['ZONE'].nunique()
# centroid[['ZONE','VTA_TAZ']].to_csv("rel.csv")
# rel = centroid[['ZONE','VTA_TAZ']]
# missing_zones = pd.DataFrame([{'ZONE' : 1454, 'VTA_TAZ' : 2786},{'ZONE':404,'VTA_TAZ':980},{'ZONE' : 190, 'VTA_TAZ' : 1890}])
# rel = pd.concat([rel,missing_zones])
# rel.to_csv('rel_vta_mtc.csv',index=False)

In [ ]:
# vt = vta_taz.columns.values
# ab = abag_2010.columns.values 
# set(ab) - set(tz)
# # set(ab) - set(tz)
# # set(ab).difference(set(tz))
# set(vt).intersection(set(ab))
# set(ab) - set(vt)

In [ ]:
rel = pd.read_csv('rel_vta_mtc.csv')

In [ ]:
rel = rel.loc[~rel['ZONE'].isnull(),]

In [ ]:
len(rel)
#Should be 2576

In [ ]:
abag_2010.columns

In [ ]:
#Join the VTA shapefile to the abag 2010 dataset!

vt_calc = pd.merge(pd.merge(abag_2010[['ZONE','TOTHH','RESACRE','CIACRE','TOTEMP']],
                            rel,how='outer'),vta_taz[['VTA_TAZ','DIST','SDIST','CITY','COUNTY','geometry','TOTHH','TOTPOP',
                            'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                            'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                            'AGE2044','AGE4564','AGE65','SFHH','MFHH']].rename(columns={"TOTHH":"VTA_TOTHH","TOTPOP":"VTA_TOTPOP",
                            "HHPOP":"VTA_HHPOP","EMPRES":"VTA_EMPRES","RESACRE":'VTA_RESACRE',"CIACRE":"VTA_CIACRE","TEMP":"VTA_TEMP"}))

In [ ]:
vt_calc.head()
# vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTPOP'].apply(lambda x: x / x.sum())
# vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTPOP'].sum()

In [ ]:
vt_tothh = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTHH'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

vt_resacre = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_RESACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
vt_ciacre = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_CIACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
vt_temp = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TEMP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

In [ ]:
vt_tothh = vt_tothh.rename(columns={"VTA_TOTHH":"VTA_TOTHH_share"})

vt_resacre = vt_resacre.rename(columns={"VTA_RESACRE":"VTA_RESACRE_share"})
vt_ciacre = vt_ciacre.rename(columns={"VTA_CIACRE":"VTA_CIACRE_share"})
vt_temp = vt_temp.rename(columns={"VTA_TEMP":"VTA_TEMP_share"})

In [ ]:
vta_final= pd.merge(pd.merge(pd.merge(pd.merge(vt_calc,vt_tothh),vt_resacre),vt_ciacre),vt_temp)

In [ ]:
vta_final['abag_2010_TOTHH_dist'] = round(vta_final['TOTHH']*vta_final['VTA_TOTHH_share'])

vta_final['abag_2010_RESACRE_dist'] = round(vta_final['RESACRE']*vta_final['VTA_RESACRE_share'])
vta_final['abag_2010_CIACRE_dist'] = round(vta_final['CIACRE']*vta_final['VTA_CIACRE_share'])
vta_final['abag_2010_TEMP_dist'] = round(vta_final['TOTEMP']*vta_final['VTA_TEMP_share'])

In [ ]:
vta_final['abag_2010_HHPOP_dist'] = round(vta_final['VTA_HHPOP']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [ ]:
vta_final['abag_2010_TOTPOP_dist'] = round(vta_final['VTA_TOTPOP']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [ ]:
vta_final['abag_2010_EMPRES_dist'] = round(vta_final['VTA_EMPRES']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [ ]:
vta_final['abag_2010_HH1_dist'] = round(vta_final['HH1']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH2_dist'] = round(vta_final['HH2']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH3_dist'] = round(vta_final['HH3']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH4_dist'] = round(vta_final['HH4']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])


In [ ]:
vta_final['abag_2010_RETEMP_dist'] = round(vta_final['RETEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_SEREMP_dist'] = round(vta_final['SEREMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_OTHEMP_dist'] = round(vta_final['OTHEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_AGEMP_dist'] = round(vta_final['AGEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_MANEMP_dist'] = round(vta_final['MANEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_WHOEMP_dist'] = round(vta_final['WHOEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])

In [ ]:
vta_final['abag_2010_AGE0004_dist'] = round(vta_final['AGE0004']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE0519_dist'] = round(vta_final['AGE0519']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE2044_dist'] = round(vta_final['AGE2044']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE4564_dist'] = round(vta_final['AGE4564']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE65_dist'] = round(vta_final['AGE65']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])

In [ ]:
vta_final['abag_2010_SFHH_dist'] = round(vta_final['SFHH']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_MFHH_dist'] = round(vta_final['MFHH']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [ ]:
vta_final

In [ ]:
vta_final.columns

In [ ]:
geometry = vta_final['geometry']
crs = mtc_taz.crs
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)

In [ ]:
geo_df.head()
geo_df.to_file('abag_2010_vta_dist')

In [ ]:
del vta_final['geometry']
vta_final.to_csv('vta_final_abag_2010.csv',index=False)
vta_final.to_excel('vta_final_abag_2010.xlsx',index=False)